In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as lite
from sqlite3 import Error
from pathlib import Path
from datetime import date
import numpy as np
import matplotlib.ticker as tick
import requests
import difflib as diff
import re 
import csv
import ast
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
%matplotlib inline

# pd.set_option('mode.chained_assignment', None)

def create_connection(db_file):
    """
    create a connection to sqlite3 database
    """
    conn = None
    try:
        conn = lite.connect(db_file, timeout=10)  # connection via sqlite3
        # engine = sa.create_engine('sqlite:///' + db_file)  # connection via sqlalchemy
        # conn = engine.connect()
    except Error as e:
        print(e)
    return conn

conn = create_connection("../CVEfixes.db")

In [ ]:
df_php = pd.read_sql_query("""SELECT m.method_change_id, m.name, m.code, m.before_change, f.programming_language
from file_change f, method_change m
WHERE m.file_change_id=f.file_change_id
AND f.programming_language='JavaScript';
""", conn)
df_php

In [ ]:
df_php = df_php.drop(df_php.columns[[0,1,4]], axis=1)

In [ ]:
df_php = df_php.rename(columns={'before_change': 'label', 'code': 'text'})
df_php

In [ ]:
df_php.loc[df_php.label == 'False', 'label'] = 0
df_php.loc[df_php.label == 'True', 'label'] = 1

In [ ]:
df_php

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_php, test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
train

In [ ]:
test

In [ ]:
train.to_json('js_train.json', orient='records')
test.to_json('js_test.json', orient='records')